**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/gym_plannable.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
%matplotlib inline
from gym_plannable.env import MazeEnv

## Gym prostredia

V prvom notebook-u zo sekcie o učení s odmenou (reinforcement learning; RL) sa ešte nebudeme venovať žiadnej konkrétnej metóde učenia s odmenou, ale tomu, ako sa reprezentujú úlohy učenia s odmenou pomocou rozhrania OpenAI Gym (ktoré sa v posledných niekoľkých rokoch stalo v istom zmysle štandardom).

[Gym od OpenAI](https://gym.openai.com/) je python-ový balíček s množstvom rôznych RL prostredí, pričom všetky sú vybavené tým istým unifikovaným rozhraním. Hlavné vlastnosti tohto rozhrania budeme ilustrovať na jednoduchom prostredí mriežkového sveta (gridworld).

### Rozhranie

#### Resetovanie a renderovanie

Resetovanie stavu prostredia a získanie jeho pozorovania, sa realizuje pomocou metódy `reset`. Táto metóda navracia:

* **obs** : počiatočné pozorovanie;
* **info** : slovník obsahujúci ďalšie informácie závisiace od konkrétneho typu prostredia;


In [ ]:
env = MazeEnv()
env.reset()

Vizuálnu reprezentáciu aktuálneho stavu prostredia vieme získať tak, že pri konštrukcii prostrediu pošleme parameter `render_mode='human'`.

Rôzne potredia podporujú rôzne režimy vizualizácie – zistiť, ktoré režimy prostredie podporuje, je možné pomocou atribútu triedy prostredia `.metadata`: napr. tu by sme mohli použiť `MazeEnv.metadata`.



In [ ]:
env = MazeEnv(render_mode='human')
env.reset();

Ako vidno, v našom prípade ide o jednoduché prostredia mriežkového sveta, kde agent je reprezentovaný modrým krúžkom, zažiatočné a cieľové stavy sú uznačené `S` a `G` (v tomto poradí) a sivé štvorčeky predstavujú múry.

#### Priestor pozorovaní a priestor akcií

S každým prostredím sa spája určitý priestor pozorovaní (atribút `observation_space`) a priestor akcií (atribút `action_space`); ide o priestory z ktorých sa berú pozorovania a akcie.

Napríklad v našom jednoduchom bludisku je stav úplne opísaný pozíciou agenta (nič iné sa nemení). Ak vezmene do úvahy, že sa agent pohybuje po mriežke $10 \times 10$, pozorovanie, ktoré navráti `reset` bude tvorené dvojicou celých čísel z rozsahu $\{0, 1, ..., 9 \}$. V rámci OpenAI gym to zodpovedá nasledujúcemu priestoru pozorovaní:



In [ ]:
env.observation_space

Priestor akcií sa skladá zo štyroch akcií:



In [ ]:
env.action_space

V našom prípade majú takýto význam:

* **0:**  hore;
* **1:**  dolu;
* **2:**  doľava;
* **3:**  doprava;
avšak tento význam, samozrejme, nie je súčasťou definície priestoru akcií.

Ak by sme chceli získať (rovnomerne) náhodnú akciu z priestoru akcií, môžeme zavolať metódu `sample`:



In [ ]:
env.action_space.sample()

#### Voľba akcií

Akcie sa volia pomocou metódy `step`; vstupným argumentom je daná akcia. Metóda `step` navracia n-ticu s nasledujúcimi prvkami:

* **obs** : nové pozorovanie;
* **reward** : okamžitá odmena spojená s daným stavovým prechodom;
* **terminated** : boolovská hodnota indikujúca či sa prostredie ukončilo (t.j. či sa dosiahol koncový stav a prostredie treba resetovať);
* **truncated** : boolovská hodnota indikujúca (na rozdiel od terminated), že prostredie bolo ukončené predčasne, napr. kvôli chybe, preto, že sa dosiahol limit na počet krokov a pod. Aj v tomto prípade je potrebné prostredie resetovať, aby bolo možné pokračovať;
* **info** : slovník obsahujúci ďalšie informácie závisiace od konkrétneho typu prostredia;


In [ ]:
env = MazeEnv()
env.reset()

obs, reward, terminated, truncated, info = env.step(0)

env.render()
print(f"observation: {obs}\nreward: {reward}\nterminated: {terminated}\ntruncated: {truncated}")

---
#### Úloha 1: Dostať agenta do cieľového stavu

**V nasledujúcej bunke zostrojte bludisko a udajte mu takú postupnosť akcií, aby agent prešiel cestu z počiatočného do cieľového stavu. V každom kroku zavolajte funkciu render, aby bolo možné pozorovať správanie sa agenta.** 

---


In [ ]:
env = MazeEnv(render_mode='human')
env.reset()


# --- env.step(     )
# --- env.render()


### Plánovateľné prostredia

V rámci mriežkových svetov, s ktorými budeme pracovať v niekoľkých nasledujúcich notebook-och poskytujeme rozšírenie oproti štandardnému OpenAI Gym rozhraniu – plánovateľné prostredia. V plánovateľnom prostredí získavate v podstate prístup k distribučnému modelu prostredia: vieme zistiť, ktoré akcie sú legálne v danom stave; aké sú pri jednotlivých akciách možné stavové prechody a aké sú ich zodpovedajúce pravdepodobnosti; vieme simulovať správanie sa prostredia po dobu viacerých časových krokov bez toho, aby sme skutočne zmenili jeho stav.

Ako už vieme, distribučný model tohto typu si vyžadujú metódy dynamického programovania, ale dokážu ho využiť aj mnohé iné prístupy, preto sa pozrime, ako vyzerá spomínané rozhranie.

#### Získanie plánovateľného stavu

Ak je prostredie plánovateľné, implementuje metódu `single_plannable_state` (alebo metódu `plannable_state`, ktorú by sme však potrebovali len keby sme pracovali s multiagentovým prostredím). Táto metóda navracia aktuálny stav prostredia ako plánovateľný stav:



In [ ]:
env = MazeEnv(render_mode='human')
env.reset()

state = env.single_plannable_state()

#### Legálne akcie

Aby sme zistili, ktoré akcie sú legálne v aktuálnom stave, používame metódu `legal_actions`:



In [ ]:
state.legal_actions()

Tu sú, striktne povedané, legálne len dve akcie, pretože agent sa nachádza v rohu: nemôže sa pohnúť smerom doľava alebo dolu.

#### Simulácia prechodov

Ak chceme, pri predpoklade akcie `a`, iterovať po všetkých možných nasledujúcich stavoch a ich zodpovedajúcich pravdepodobnostiach, môžeme použiť metódu `all_next(a)`:



In [ ]:
a = state.legal_actions()[0]

for next_state, prob in state.all_next(a):
    print(f"observation {next_state.observation()}; " + 
          f"reward: {next_state.reward()}; probability: {prob}")

Ak máme záujem len o jednu vzorku prechodu, dá sa použiť `next(a)`:



In [ ]:
a = state.legal_actions()[0]
next_state = state.next(a)
print(f"observation {next_state.observation()}; " + 
      f"reward: {next_state.reward()}; probability: {prob}")

#### Príznak done

Aby sme zistili, či je stav konečný, môžeme zase použiť metódu `is_done`:



In [ ]:
state.is_done()